In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("results_tabz.csv")
df = df[df["split_method"] == "random_split"]

In [3]:
rename_map = {
    'ConstantPredictor': 'Const.',
    'FTTransformer': 'FT-Trans.',
    'LGBMRegressor': 'GBT',
    'LGBMClassifier': 'GBT',
    'LinearRegressor': 'Lin. Regr.',
    'RandomForestRegressor': 'RF',
    'RandomForestClassifier': 'RF',
    'ResNet': 'ResNet',
    'TabPFNRegressor': 'TabPFN',
    'TabPFNClassifier': 'TabPFN',
    'LogisticRegressor': 'Log. Regr.',
    'GPBoost_LogLoss': 'GPBoost',
    'GPBoost_Accuracy': 'GPBoost',
    'GPBoost_CRPS': 'GPBoost',
    'GPBoost_RMSE': 'GPBoost'

}


In [4]:
rename_map_rmse = {
    'ConstantPredictor': 'Const.',
    'FTTransformer': 'FT-Trans.',
    'LGBMRegressor': 'GBT',
    'LinearRegressor': 'Lin. Regr.',
    'RandomForestRegressor': 'RF',
    'ResNet': 'ResNet',
    'TabPFNRegressor': 'TabPFN',
    'LogisticRegressor': 'Log. Regr.',
    'GPBoost_RMSE': 'GPBoost'}

Average Difference and Average Relative Difference RMSE

In [5]:
metric = "RMSE"
dfm   = df[df["metric"] == metric]

pivot_rmse = dfm.pivot_table(
    index="task_id",
    columns="model",
    values="value",
    aggfunc="mean"
)
pivot_rmse = pivot_rmse.rename(columns=rename_map_rmse)

pivot_rmse.to_csv(f"avg_{metric}_per_task_per_model_IP_tabz.csv", float_format="%.3f")


In [6]:
def avg_relative_diff(pivot: pd.DataFrame, error_metric: bool = True) -> pd.Series:

    if error_metric:
        best_per_task = pivot.min(axis=1)   # best is minimum
        # (v - best)/best * 100
        rel_diff = (pivot.sub(best_per_task, axis=0)
                         .div(best_per_task, axis=0)
                         * 100)
    else:
        best_per_task = pivot.max(axis=1)   # best is maximum
        # (best - v)/best * 100
        rel_diff = (best_per_task.sub(pivot, axis=0)
                              .div(best_per_task, axis=0)
                              * 100)

    # average across tasks (skipping any NaNs)
    return rel_diff.mean(axis=0)


avg_diff_rmse = avg_relative_diff(pivot_rmse, error_metric=True)

print("Average relative difference (RMSE) - Interpolation :")
print(avg_diff_rmse.round(2))


Average relative difference (RMSE) - Interpolation :
Series([], dtype: float64)


In [7]:
def normalized_accuracy(pivot: pd.DataFrame,
                        error_metric: bool = True) -> pd.Series:

    if error_metric:
        best = pivot.min(axis=1)  

        mid  = pivot.apply(
            lambda s: s.dropna().nlargest(3).min(),
            axis=1
        )
        norm = (
            pivot
            .rsub(mid,       axis=0)    # mid[t] - pivot.loc[t, m]
            .div(mid - best, axis=0)
            .clip(0, 1)
        )
    else:
        best = pivot.max(axis=1)
        mid  = pivot.apply(
            lambda s: s.dropna().nsmallest(3).max(),
            axis=1
        )
        # 3) norm_acc per cell = (acc - mid) / (best - mid), clipped
        norm = (
            pivot
            .rsub(mid,          axis=0)   # pivot.loc[t,m] - mid[t]
            .mul(-1)
            .div(mid - best, axis=0)
            .clip(0, 1)
        )

    return norm.mean(axis=0)


avg_norm_acc_rmse = normalized_accuracy(pivot_rmse, error_metric=True)
print("Average normalized accuracy (from RMSE) - Interpolation:")
print((100 * avg_norm_acc_rmse).round(2))


Average normalized accuracy (from RMSE) - Interpolation:
Series([], dtype: float64)


In [8]:
def avg_rank(pivot: pd.DataFrame, error_metric: bool = True) -> pd.Series:
  
    ranks = pivot.rank(
        axis=1,
        method="average",   
        ascending=error_metric
    )

    return ranks.mean(axis=0)

avg_rank_rmse = avg_rank(pivot_rmse, error_metric=True)
print("Average rank (RMSE) - Interpolation:")
print(avg_rank_rmse.round(2))



Average rank (RMSE) - Interpolation:
Series([], dtype: float64)


In [9]:
avg_diff    = avg_relative_diff(pivot_rmse, error_metric=True)           # in %
avg_acc     = normalized_accuracy(pivot_rmse, error_metric=True) * 100   # convert to % 
avg_rank    = avg_rank(pivot_rmse, error_metric=True)

metrics = pd.DataFrame(
    [avg_diff, avg_acc, avg_rank],
    index=["Avg. diff.", "Avg. acc.", "Avg. rank."]
)

pivot_with_summaries = pd.concat([pivot_rmse, metrics])

out = pivot_with_summaries.reset_index().rename(columns={"index":"task_id"})

out.to_csv(f"avg_{metric}_with_summary_IP.csv", float_format="%.3f", index=False)

latex = out.to_latex(index=False, escape=True, float_format="%.3f")
with open(f"_tabz_{metric}_IP_results.tex","w") as f:
    f.write(latex)

Average Difference and Average Relative Difference CRPS

In [10]:
rename_map_crps = {
    'ConstantPredictor': 'Const.',
    'FTTransformer': 'FT-Trans.',
    'LGBMRegressor': 'GBT',
    'LinearRegressor': 'Lin. Regr.',
    'RandomForestRegressor': 'RF',
    'ResNet': 'ResNet',
    'TabPFNRegressor': 'TabPFN',
    'LogisticRegressor': 'Log. Regr.',
    'GPBoost_CRPS': 'GPBoost',

}


In [11]:
metric1 = "CRPS"
dfm1   = df[df["metric"] == metric1]

pivot_crps = dfm1.pivot_table(
    index="task_id",
    columns="model",
    values="value",
    aggfunc="mean"
)

pivot_crps = pivot_crps.rename(columns=rename_map_crps)
pivot_crps.to_csv(f"avg_{metric1}_per_task_per_model_IP_tabz.csv", float_format="%.3f")


In [12]:
def avg_relative_diff(pivot: pd.DataFrame, error_metric: bool = True) -> pd.Series:

    if error_metric:
        best_per_task = pivot.min(axis=1)   # best is minimum
        # (v - best)/best * 100
        rel_diff = (pivot.sub(best_per_task, axis=0)
                         .div(best_per_task, axis=0)
                         * 100)
    else:
        best_per_task = pivot.max(axis=1)   # best is maximum
        # (best - v)/best * 100
        rel_diff = (best_per_task.sub(pivot, axis=0)
                              .div(best_per_task, axis=0)
                              * 100)

    # average across tasks (skipping any NaNs)
    return rel_diff.mean(axis=0)


avg_diff_crps = avg_relative_diff(pivot_crps, error_metric=True)

print("Average relative difference (CRPS) - Interpolation:")
print(avg_diff_crps.round(2))


Average relative difference (CRPS) - Interpolation:
Series([], dtype: float64)


In [13]:
def normalized_accuracy(pivot: pd.DataFrame,
                        error_metric: bool = True) -> pd.Series:

    if error_metric:
        best = pivot.min(axis=1)  

        mid  = pivot.apply(
            lambda s: s.dropna().nlargest(3).min(),
            axis=1
        )
        norm = (
            pivot
            .rsub(mid,       axis=0)    # mid[t] - pivot.loc[t, m]
            .div(mid - best, axis=0)
            .clip(0, 1)
        )
    else:
        best = pivot.max(axis=1)
        mid  = pivot.apply(
            lambda s: s.dropna().nsmallest(3).max(),
            axis=1
        )
        # 3) norm_acc per cell = (acc - mid) / (best - mid), clipped
        norm = (
            pivot
            .rsub(mid,          axis=0)   # pivot.loc[t,m] - mid[t]
            .mul(-1)
            .div(mid - best, axis=0)
            .clip(0, 1)
        )

    return norm.mean(axis=0)


avg_norm_acc_crps = normalized_accuracy(pivot_crps, error_metric=True)
print("Average normalized accuracy (from CRPS) - Interpolation:")
print((100 * avg_norm_acc_crps).round(2))


Average normalized accuracy (from CRPS) - Interpolation:
Series([], dtype: float64)


In [14]:
def avg_rank(pivot: pd.DataFrame, error_metric: bool = True) -> pd.Series:
  
    ranks = pivot.rank(
        axis=1,
        method="average",   
        ascending=error_metric
    )

    return ranks.mean(axis=0)

avg_rank_crps = avg_rank(pivot_crps, error_metric=True)
print("Average rank (CRPS) - Interpolation:")
print(avg_rank_crps.round(2))



Average rank (CRPS) - Interpolation:
Series([], dtype: float64)


In [15]:
avg_diff1    = avg_relative_diff(pivot_crps, error_metric=True)           # in %
avg_acc1     = normalized_accuracy(pivot_crps, error_metric=True) * 100   # convert to % 
avg_rank1    = avg_rank(pivot_crps, error_metric=True)

metrics = pd.DataFrame(
    [avg_diff1, avg_acc1, avg_rank1],
    index=["Avg. diff.", "Avg. acc.", "Avg. rank."]
)

pivot_with_summaries1 = pd.concat([pivot_crps, metrics])

out1 = pivot_with_summaries1.reset_index().rename(columns={"index":"task_id"})

out1.to_csv(f"avg_{metric1}_with_summary_IP.csv", float_format="%.3f", index=False)

latex1 = out1.to_latex(index=False, escape=True, float_format="%.3f")
with open(f"tabz_{metric1}_IP_results.tex","w") as f:
    f.write(latex1)

Average Difference and Average Relative Difference LOGLOSS

In [16]:
rename_map_ll = {
    'ConstantPredictor': 'Const.',
    'FTTransformer': 'FT-Trans.',
    'LGBMClassifier': 'GBT',
    'RandomForestClassifier': 'RF',
    'ResNet': 'ResNet',
    'TabPFNClassifier': 'TabPFN',
    'LogisticRegressor': 'Log. Regr.',
    'GPBoost_LogLoss': 'GPBoost'

}


In [17]:
metric2 = "LogLoss"
dfm2   = df[df["metric"] == metric2]

pivot_ll = dfm2.pivot_table(
    index="task_id",
    columns="model",
    values="value",
    aggfunc="mean"
)
pivot_ll = pivot_ll.rename(columns=rename_map_ll)
pivot_ll.to_csv(f"avg_{metric2}_per_task_per_model_IP_tabz.csv", float_format="%.3f")


In [18]:
def avg_relative_diff(pivot: pd.DataFrame, error_metric: bool = True) -> pd.Series:

    if error_metric:
        best_per_task = pivot.min(axis=1)   # best is minimum
        # (v - best)/best * 100
        rel_diff = (pivot.sub(best_per_task, axis=0)
                         .div(best_per_task, axis=0)
                         * 100)
    else:
        best_per_task = pivot.max(axis=1)   # best is maximum
        # (best - v)/best * 100
        rel_diff = (best_per_task.sub(pivot, axis=0)
                              .div(best_per_task, axis=0)
                              * 100)

    return rel_diff.mean(axis=0)


avg_diff_ll = avg_relative_diff(pivot_ll, error_metric=True)

print("Average relative difference (LogLoss) - Interpolation:")
print(avg_diff_ll.round(2))


Average relative difference (LogLoss) - Interpolation:
model
Const.        5108.52
Engression     117.13
FT-Trans.      129.85
GPBoost         60.58
GBT            185.00
Log. Regr.      63.81
MLP             85.33
RF             185.00
ResNet          87.13
TabPFN           0.62
dtype: float64


In [19]:
def normalized_accuracy(pivot: pd.DataFrame,
                        error_metric: bool = True) -> pd.Series:

    if error_metric:
        best = pivot.min(axis=1)  

        mid  = pivot.apply(
            lambda s: s.dropna().nlargest(3).min(),
            axis=1
        )
        norm = (
            pivot
            .rsub(mid,       axis=0)    # mid[t] - pivot.loc[t, m]
            .div(mid - best, axis=0)
            .clip(0, 1)
        )
    else:
        best = pivot.max(axis=1)
        mid  = pivot.apply(
            lambda s: s.dropna().nsmallest(3).max(),
            axis=1
        )
        # 3) norm_acc per cell = (acc - mid) / (best - mid), clipped
        norm = (
            pivot
            .rsub(mid,          axis=0)   # pivot.loc[t,m] - mid[t]
            .mul(-1)
            .div(mid - best, axis=0)
            .clip(0, 1)
        )

    return norm.mean(axis=0)


avg_norm_acc_ll = normalized_accuracy(pivot_ll, error_metric=True)
print("Average normalized accuracy (from LogLoss) - Interpolation:")
print((100 * avg_norm_acc_ll).round(2))


Average normalized accuracy (from LogLoss) - Interpolation:
model
Const.         0.00
Engression    19.91
FT-Trans.     34.93
GPBoost       72.17
GBT            2.39
Log. Regr.    65.28
MLP           53.26
RF             2.39
ResNet        41.53
TabPFN        99.10
dtype: float64


In [20]:
def avg_rank(pivot: pd.DataFrame, error_metric: bool = True) -> pd.Series:
  
    ranks = pivot.rank(
        axis=1,
        method="average",   
        ascending=error_metric
    )

    return ranks.mean(axis=0)

avg_rank_ll = avg_rank(pivot_ll, error_metric=True)
print("Average rank (LogLoss) - Interpolation:")
print(avg_rank_ll.round(2))



Average rank (LogLoss) - Interpolation:
model
Const.        8.59
Engression    6.25
FT-Trans.     5.52
GPBoost       3.08
GBT           7.14
Log. Regr.    3.52
MLP           3.58
RF            7.14
ResNet        5.15
TabPFN        1.21
dtype: float64


In [21]:
avg_diff2    = avg_relative_diff(pivot_ll, error_metric=True)           # in %
avg_acc2     = normalized_accuracy(pivot_ll, error_metric=True) * 100   # convert to % 
avg_rank2    = avg_rank(pivot_ll, error_metric=True)

metrics = pd.DataFrame(
    [avg_diff2, avg_acc2, avg_rank2],
    index=["Avg. diff.", "Avg. acc.", "Avg. rank."]
)

pivot_with_summaries2 = pd.concat([pivot_ll, metrics])

out2 = pivot_with_summaries2.reset_index().rename(columns={"index":"task_id"})

out2.to_csv(f"avg_{metric2}_with_summary_IP.csv", float_format="%.3f", index=False)

latex2 = out2.to_latex(index=False, escape=True, float_format="%.3f")
with open(f"tabz_{metric2}_IP_results.tex","w") as f:
    f.write(latex2)

Average Difference and Average Relative Difference Accuracy

In [22]:
rename_map_acc = {
    'ConstantPredictor': 'Const.',
    'FTTransformer': 'FT-Trans.',
    'LGBMClassifier': 'GBT',
    'RandomForestClassifier': 'RF',
    'ResNet': 'ResNet',
    'TabPFNClassifier': 'TabPFN',
    'LogisticRegressor': 'Log. Regr.',
    'GPBoost_Accuracy': 'GPBoost'

}


In [23]:
metric3 = "Accuracy"
dfm3   = df[df["metric"] == metric3]

pivot_acc = dfm3.pivot_table(
    index="task_id",
    columns="model",
    values="value",
    aggfunc="mean"
)

pivot_acc = pivot_acc.rename(columns=rename_map_acc)
pivot_acc.to_csv(f"avg_{metric3}_per_task_per_model_IP_tabz.csv", float_format="%.5f")


In [24]:
def avg_relative_diff(pivot: pd.DataFrame, error_metric: bool = True) -> pd.Series:

    if error_metric:
        best_per_task = pivot.min(axis=1)   # best is minimum
        # (v - best)/best * 100
        rel_diff = (pivot.sub(best_per_task, axis=0)
                         .div(best_per_task, axis=0)
                         * 100)
    else:
        best_per_task = pivot.max(axis=1)   # best is maximum
        # (best - v)/best * 100
        rel_diff = (pivot.sub(best_per_task, axis=0)     
                         .mul(-1)                         
                         .div(best_per_task , axis=0)
                         * 100)

    return rel_diff.mean(axis=0)


avg_diff_accuracy = avg_relative_diff(pivot_acc, error_metric=False)

print("\nAverage relative difference (Accuracy) - Interpolation:")
print(avg_diff_accuracy.round(2))



Average relative difference (Accuracy) - Interpolation:
model
Const.        36.47
Engression    58.05
FT-Trans.     33.92
GPBoost        7.79
GBT            3.86
Log. Regr.     9.71
MLP           19.57
RF             7.72
ResNet        25.94
TabPFN         2.39
dtype: float64


In [25]:
def normalized_accuracy(pivot: pd.DataFrame,
                        error_metric: bool = True) -> pd.Series:

    if error_metric:
        best = pivot.min(axis=1)  

        mid  = pivot.apply(
            lambda s: s.dropna().nlargest(3).min(),
            axis=1
        )
        norm = (
            pivot
            .rsub(mid,       axis=0)    # mid[t] - pivot.loc[t, m]
            .div(mid - best, axis=0)
            .clip(0, 1)
        )
    else:
        best = pivot.max(axis=1)
        mid  = pivot.apply(
            lambda s: s.dropna().nsmallest(3).max(),
            axis=1
        )
        # 3) norm_acc per cell = (acc - mid) / (best - mid), clipped
        norm = (
            pivot
            .rsub(mid,          axis=0)   # pivot.loc[t,m] - mid[t]
            .div(mid - best, axis=0)
            .clip(0, 1)
        )

    return norm.mean(axis=0)


avg_norm_acc_acc = normalized_accuracy(pivot_acc, error_metric=False)
print("Average normalized accuracy (from Accuracy) - Interpolation:")
print((100 * avg_norm_acc_acc).round(3))


Average normalized accuracy (from Accuracy) - Interpolation:
model
Const.         8.150
Engression     0.000
FT-Trans.     28.621
GPBoost       66.743
GBT           80.966
Log. Regr.    56.439
MLP           42.072
RF            68.774
ResNet        31.625
TabPFN        90.043
dtype: float64


In [26]:
def avg_rank(pivot: pd.DataFrame, error_metric: bool = True) -> pd.Series:
  
    ranks = pivot.rank(
        axis=1,
        method="average",   
        ascending=error_metric
    )

    return ranks.mean(axis=0)

avg_rank_acc = avg_rank(pivot_acc, error_metric=False)
print("Average rank (Accuracy) -Interpolation:")
print(avg_rank_acc.round(2))



Average rank (Accuracy) -Interpolation:
model
Const.        7.69
Engression    8.90
FT-Trans.     6.78
GPBoost       3.85
GBT           3.22
Log. Regr.    4.58
MLP           5.53
RF            3.95
ResNet        6.13
TabPFN        2.02
dtype: float64


In [27]:
avg_diff3    = avg_relative_diff(pivot_acc, error_metric=False)           # in %
avg_acc3     = normalized_accuracy(pivot_acc, error_metric=False) * 100   # convert to % 
avg_rank3    = avg_rank(pivot_acc, error_metric=False)

metrics = pd.DataFrame(
    [avg_diff3, avg_acc3, avg_rank3],
    index=["Avg. diff.", "Avg. acc.", "Avg. rank."]
)

pivot_with_summaries3 = pd.concat([pivot_acc, metrics])

out3 = pivot_with_summaries3.reset_index().rename(columns={"index":"task_id"})

out3.to_csv(f"avg_{metric3}_with_summary_IP.csv", float_format="%.3f", index=False)

latex3 = out3.to_latex(index=False, escape=True, float_format="%.3f")
with open(f"tabz_{metric3}_IP_results.tex","w") as f:
    f.write(latex3)